Baseball Prediction: 3a - Getting Odds Data

In this notebook, we will get historical odds data from oddsshark.com. We will use the pandas read_html function to grab a table into a dataframe, and show how to programmatically sweep through all the necessary urls to get the data we need.

We will save this data as a collection of csv files. In the next notebook, we will use these csv files to add the odds information to our primary dataframe.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
import lxml
import html5lib
from urllib.request import urlopen
import time

In [3]:
df1 = pd.read_html('https://www.oddsshark.com/stats/gamelog/baseball/mlb/27000?season=2021')[0]
df1.head(10)

,Date,Opponent,Game,Result,Score,Line,OU,Total
0,"Apr 1, 2021",vs St. Louis,REG,L,11-6,-123,O,7.0
1,"Apr 3, 2021",vs St. Louis,REG,W,9-6,-120,O,9.0
2,"Apr 4, 2021",vs St. Louis,REG,W,12-1,-101,O,10.0
3,"Apr 5, 2021",vs Pittsburgh,REG,W,5-3,-145,U,9.0
4,"Apr 6, 2021",vs Pittsburgh,REG,W,14-1,-155,O,9.0
5,"Apr 7, 2021",vs Pittsburgh,REG,W,11-4,-220,O,8.0
6,"Apr 9, 2021",@ Arizona,REG,W,6-5,-148,O,9.5
7,"Apr 10, 2021",@ Arizona,REG,L,8-3,-125,O,9.5
8,"Apr 11, 2021",@ Arizona,REG,L,7-0,-112,U,9.0
9,"Apr 12, 2021",@ San Francisco,REG,W,3-0,-103,U,8.5


In [4]:
def line_to_prob(line):
    prob_underdog = 100/(np.abs(line)+100) # this is the probability for the 
    add_term = ((1-np.sign(line))/2) # 0 if negative, 1 if positive
    mult_factor = np.sign(line) # -1 if negative, 1 if positive
    # if line is positive, team is underdog, give 0 + 1*prob_underdog
    # if line is negative, team is favorited give 1 + (-1)*prob_underdog
    imp_prob = add_term + mult_factor * prob_underdog 
    return(imp_prob)


Plan of Attack

- Get the "number" for each team
- Read the table for each team, and for each season (2019-2022)
- Lightly process the data frame (remove playoffs, process date, add game_number, add "source_team", convert line)
- Save each file

In [5]:
# manually figure out what number in url corresponds to which team
# use the 3 letter abbrev from retrosheet for each team

oddsshark_num_to_team_dict = {}
oddsshark_num_to_team_dict[26995]='PHI'
oddsshark_num_to_team_dict[26996]='SDN'
oddsshark_num_to_team_dict[26997]='SFN'
oddsshark_num_to_team_dict[26998]='ANA'
oddsshark_num_to_team_dict[26999]='DET'
oddsshark_num_to_team_dict[27000]='CIN'
oddsshark_num_to_team_dict[27001]='NYA'
oddsshark_num_to_team_dict[27002]='TEX'
oddsshark_num_to_team_dict[27003]='TBA'
oddsshark_num_to_team_dict[27004]='COL'
oddsshark_num_to_team_dict[27005]='MIN'
oddsshark_num_to_team_dict[27006]='KCA'
oddsshark_num_to_team_dict[27007]='ARI'
oddsshark_num_to_team_dict[27008]='BAL'
oddsshark_num_to_team_dict[27009]='ATL'
oddsshark_num_to_team_dict[27010]='TOR'
oddsshark_num_to_team_dict[27011]='SEA'
oddsshark_num_to_team_dict[27012]='MIL'
oddsshark_num_to_team_dict[27013]='PIT'
oddsshark_num_to_team_dict[27014]='NYN'
oddsshark_num_to_team_dict[27015]='LAN'
oddsshark_num_to_team_dict[27016]='OAK'
oddsshark_num_to_team_dict[27017]='WAS'
oddsshark_num_to_team_dict[27018]='CHA'
oddsshark_num_to_team_dict[27019]='SLN'
oddsshark_num_to_team_dict[27020]='CHN'
oddsshark_num_to_team_dict[27021]='BOS'
oddsshark_num_to_team_dict[27022]='MIA'
oddsshark_num_to_team_dict[27023]='HOU'
oddsshark_num_to_team_dict[27024]='CLE'

In [7]:
for i in range(26995, 27025):
    team_name = oddsshark_num_to_team_dict[i]
    print(team_name)
    for season in range(2019,2023):
        print(season)
        url = 'https://www.oddsshark.com/stats/gamelog/baseball/mlb/'+str(i)+'?season='+str(season)
        df_temp = pd.read_html(url)[0]
        df_temp = df_temp[df_temp.Game=='REG']
        print(df_temp.shape)
        df_temp['team_source'] = team_name
        df_temp['season'] = season
        df_temp['date_numeric'] = pd.to_datetime(df_temp.Date).astype(str).str.replace('-','')
        df_temp['game_no'] = np.arange(1,df_temp.shape[0]+1)
        df_temp['prob_implied'] = line_to_prob(df_temp['Line'])      
        next_game_date = np.concatenate((df_temp['date_numeric'].iloc[1:],[0]))
        previous_game_date = np.concatenate(([0], df_temp['date_numeric'].iloc[:-1]))
        game_1_dblheader = (df_temp.date_numeric.to_numpy()==next_game_date).astype(int)
        game_2_dblheader = (df_temp.date_numeric.to_numpy()==previous_game_date).astype(int)*2
        df_temp['dblheader_num'] = game_1_dblheader+game_2_dblheader        
        fname_out = 'oddsshark_'+team_name+'_'+str(season)+'.csv'
        df_temp.to_csv('/Users/gilliancurtis/Desktop/beatingVegas/oddshark/'+fname_out,index=False)
        time.sleep(0.5)

PHI
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
SDN
2019
(162, 8)
2020
(60, 8)
2021
(161, 8)
2022
(162, 8)
SFN
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
ANA
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
DET
2019
(161, 8)
2020
(58, 8)
2021
(162, 8)
2022
(162, 8)
CIN
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
NYA
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
TEX
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
TBA
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
COL
2019
(162, 8)
2020
(60, 8)
2021
(161, 8)
2022
(162, 8)
MIN
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
KCA
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
ARI
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
BAL
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
ATL
2019
(162, 8)
2020
(60, 8)
2021
(160, 8)
2022
(162, 8)
TOR
2019
(162, 8)
2020
(60, 8)
2021
(162, 8)
2022
(162, 8)
SEA
2019
(162, 8)
2020
(60, 8)
2021
(161, 8)
2022
(162, 